In [1]:
import pandas as pd
import numpy as np
import re
from collections import defaultdict

import psycopg2 as pg
connection_args = {'host':'localhost', 
                   'user': , 
                   'dbname':'worldbank', 
                   'port':5432}

conn = pg.connect(**connection_args)
cur = conn.cursor()

from tqdm import tqdm
tqdm.pandas()

pd.set_option('display.float_format', lambda x: '%.5f' % x)

/Users/seansisler/opt/anaconda3/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


### Import data and create DataFrames for other tables for Postgres

In [2]:
indicators = pd.read_csv('indicators.csv')
print(indicators.shape)
indicators.head()

(5656458, 6)


,CountryName,CountryCode,IndicatorName,IndicatorCode,Year,Value
0,Arab World,ARB,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1960,133.56091
1,Arab World,ARB,Age dependency ratio (% of working-age populat...,SP.POP.DPND,1960,87.79760
2,Arab World,ARB,"Age dependency ratio, old (% of working-age po...",SP.POP.DPND.OL,1960,6.63458
3,Arab World,ARB,"Age dependency ratio, young (% of working-age ...",SP.POP.DPND.YG,1960,81.02333
4,Arab World,ARB,Arms exports (SIPRI trend indicator values),MS.MIL.XPRT.KD,1960,3000000.00000


In [3]:
countries_regions = indicators[['CountryName', 'CountryCode']].drop_duplicates()
countries_regions.reset_index(drop=True, inplace=True)
print(countries_regions.shape)
countries_regions.head()

(247, 2)


,CountryName,CountryCode
0,Arab World,ARB
1,Caribbean small states,CSS
2,Central Europe and the Baltics,CEB
3,East Asia & Pacific (all income levels),EAS
4,East Asia & Pacific (developing only),EAP


In [4]:
income_groups = pd.read_excel('income-groups-historical.xlsx')
income_groups.head(10)

,Unnamed: 0,Bank's fiscal year:,FY89,FY90,FY91,FY92,FY93,FY94,FY95,FY96,...,FY12,FY13,FY14,FY15,FY16,FY17,NaN,nan.1,nan.2,nan.3
0,NaN,Data for calendar year :,1987,1988,1989,1990,1991,1992,1993,1994,...,2010,2011,2012,2013,2014,2015,NaN,NaN,NaN,NaN
1,NaN,Low income (L),<= 480,<= 545,<= 580,<= 610,<= 635,<= 675,<= 695,<= 725,...,"<= 1,005","<= 1,025","<= 1,035","<= 1,045","<= 1,045","<= 1,025",NaN,NaN,NaN,NaN
2,NaN,Lower middle income (LM),"481-1,940","546-2,200","581-2,335","611-2,465","636-2,555","676-2,695","696-2,785","726-2,895",...,"1,006-3,975","1,026-4,035","1,036-4,085","1,046-4,125","1,046-4,125","1,026-4,035",NaN,NaN,NaN,NaN
3,NaN,Upper middle income (UM),"1,941-6,000","2,201-6,000","2,336-6,000","2,466-7,620","2,556-7,910","2,696-8,355","2,786-8,625","2,896-8,955",...,"3,976-12,275","4,036-12,475","4,086-12,615","4,126-12,745","4,126-12,735","4,036-12,475",NaN,NaN,NaN,NaN
4,NaN,High income (H),"> 6,000","> 6,000","> 6,000","> 7,620","> 7,910","> 8,355","> 8,625","> 8,955",...,"> 12,275","> 12,475","> 12,615","> 12,745","> 12,735","> 12,475",NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,AFG,Afghanistan,L,L,L,L,L,L,L,L,...,L,L,L,L,L,L,L,L,L,L
7,ALB,Albania,..,..,..,LM,LM,LM,L,L,...,UM,LM,UM,UM,UM,UM,UM,UM,UM,UM
8,DZA,Algeria,UM,UM,LM,LM,LM,LM,LM,LM,...,UM,UM,UM,UM,UM,UM,UM,UM,UM,LM
9,ASM,American Samoa,H,H,H,UM,UM,UM,UM,UM,...,UM,UM,UM,UM,UM,UM,UM,UM,UM,UM


In [5]:
income_group_cutoffs = income_groups.iloc[:5]
income_group_cutoffs

,Unnamed: 0,Bank's fiscal year:,FY89,FY90,FY91,FY92,FY93,FY94,FY95,FY96,...,FY12,FY13,FY14,FY15,FY16,FY17,NaN,nan.1,nan.2,nan.3
0,NaN,Data for calendar year :,1987,1988,1989,1990,1991,1992,1993,1994,...,2010,2011,2012,2013,2014,2015,NaN,NaN,NaN,NaN
1,NaN,Low income (L),<= 480,<= 545,<= 580,<= 610,<= 635,<= 675,<= 695,<= 725,...,"<= 1,005","<= 1,025","<= 1,035","<= 1,045","<= 1,045","<= 1,025",NaN,NaN,NaN,NaN
2,NaN,Lower middle income (LM),"481-1,940","546-2,200","581-2,335","611-2,465","636-2,555","676-2,695","696-2,785","726-2,895",...,"1,006-3,975","1,026-4,035","1,036-4,085","1,046-4,125","1,046-4,125","1,026-4,035",NaN,NaN,NaN,NaN
3,NaN,Upper middle income (UM),"1,941-6,000","2,201-6,000","2,336-6,000","2,466-7,620","2,556-7,910","2,696-8,355","2,786-8,625","2,896-8,955",...,"3,976-12,275","4,036-12,475","4,086-12,615","4,126-12,745","4,126-12,735","4,036-12,475",NaN,NaN,NaN,NaN
4,NaN,High income (H),"> 6,000","> 6,000","> 6,000","> 7,620","> 7,910","> 8,355","> 8,625","> 8,955",...,"> 12,275","> 12,475","> 12,615","> 12,745","> 12,735","> 12,475",NaN,NaN,NaN,NaN


In [6]:
income_groups = income_groups.iloc[6:]
income_groups.rename(columns={'Unnamed: 0':'country_code', 
                              "Bank's fiscal year:":'country_name'}, inplace=True)
income_groups.columns = list(income_groups.columns[:2]) + [year for year in range(1987, 2020)]
income_groups.reset_index(drop=True, inplace=True)
print(income_groups.shape)
income_groups.head()

(218, 35)


,country_code,country_name,1987,1988,1989,1990,1991,1992,1993,1994,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,AFG,Afghanistan,L,L,L,L,L,L,L,L,...,L,L,L,L,L,L,L,L,L,L
1,ALB,Albania,..,..,..,LM,LM,LM,L,L,...,UM,LM,UM,UM,UM,UM,UM,UM,UM,UM
2,DZA,Algeria,UM,UM,LM,LM,LM,LM,LM,LM,...,UM,UM,UM,UM,UM,UM,UM,UM,UM,LM
3,ASM,American Samoa,H,H,H,UM,UM,UM,UM,UM,...,UM,UM,UM,UM,UM,UM,UM,UM,UM,UM
4,AND,Andorra,..,..,..,H,H,H,H,H,...,H,H,H,H,H,H,H,H,H,H


In [7]:
income_group_cutoffs.drop(columns=['Unnamed: 0'], inplace=True)
income_group_cutoffs.iloc[0, -1] = 2019
income_group_cutoffs.iloc[0, -2] = 2018
income_group_cutoffs.iloc[0, -3] = 2017
income_group_cutoffs.iloc[0, -4] = 2016
income_group_cutoffs.columns = income_group_cutoffs.iloc[0]
income_group_cutoffs.drop(0, inplace=True)
income_group_cutoffs.drop([2016, 2017, 2018, 2019], axis=1, inplace=True)
income_group_cutoffs.head()

,Data for calendar year :,1987,1988,1989,1990,1991,1992,1993,1994,1995,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
1,Low income (L),<= 480,<= 545,<= 580,<= 610,<= 635,<= 675,<= 695,<= 725,<= 765,...,<= 905,<= 935,<= 975,<= 995,"<= 1,005","<= 1,025","<= 1,035","<= 1,045","<= 1,045","<= 1,025"
2,Lower middle income (LM),"481-1,940","546-2,200","581-2,335","611-2,465","636-2,555","676-2,695","696-2,785","726-2,895","766-3,035",...,"906-3,595","936-3,705","976-3,855","996-3,945","1,006-3,975","1,026-4,035","1,036-4,085","1,046-4,125","1,046-4,125","1,026-4,035"
3,Upper middle income (UM),"1,941-6,000","2,201-6,000","2,336-6,000","2,466-7,620","2,556-7,910","2,696-8,355","2,786-8,625","2,896-8,955","3,036-9,385",...,"3,596-11,115","3,706-11,455","3,856-11,905","3,946-12,195","3,976-12,275","4,036-12,475","4,086-12,615","4,126-12,745","4,126-12,735","4,036-12,475"
4,High income (H),"> 6,000","> 6,000","> 6,000","> 7,620","> 7,910","> 8,355","> 8,625","> 8,955","> 9,385",...,"> 11,115","> 11,455","> 11,905","> 12,195","> 12,275","> 12,475","> 12,615","> 12,745","> 12,735","> 12,475"


In [8]:
country_codes_incomes = income_groups.country_code.tolist()

In [9]:
def mismatch_or_missing_country_codes(row):
    if row['CountryCode'] in country_codes_incomes:
        country_codes_incomes.pop(country_codes_incomes.index(row['CountryCode']))
        return np.nan
    return row['CountryName'], row['CountryCode']

In [10]:
mismatch = countries_regions.apply(mismatch_or_missing_country_codes, axis=1).dropna()

In [11]:
mismatch

0                                      (Arab World, ARB)
1                          (Caribbean small states, CSS)
2                  (Central Europe and the Baltics, CEB)
3         (East Asia & Pacific (all income levels), EAS)
4           (East Asia & Pacific (developing only), EAP)
5                                       (Euro area, EMU)
6       (Europe & Central Asia (all income levels), ECS)
7         (Europe & Central Asia (developing only), ECA)
8                                  (European Union, EUU)
9        (Fragile and conflict affected situations, FCS)
10         (Heavily indebted poor countries (HIPC), HPC)
11                                    (High income, HIC)
12                           (High income: nonOECD, NOC)
13                              (High income: OECD, OEC)
14     (Latin America & Caribbean (all income levels)...
15     (Latin America & Caribbean (developing only), ...
16     (Least developed countries: UN classification,...
17                            (

In [12]:
country_codes_incomes

['AND', 'VGB', 'COD', 'GIB', 'IMN', 'XKX', 'NRU', 'ROU', 'TWN', 'TLS', 'PSE']

In [13]:
income_groups[income_groups.country_code.str.contains('(AND|VGB|COD|GIB|IMN|XKX|NRU|ROU|TWN|TLS|PSE)')]

/Users/seansisler/opt/anaconda3/lib/python3.7/site-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,country_code,country_name,1987,1988,1989,1990,1991,1992,1993,1994,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
4,AND,Andorra,..,..,..,H,H,H,H,H,...,H,H,H,H,H,H,H,H,H,H
27,VGB,British Virgin Islands,..,..,..,..,..,..,..,..,...,..,..,..,..,..,H,H,H,H,H
44,COD,"Congo, Dem. Rep.",L,L,L,L,L,L,L,L,...,L,L,L,L,L,L,L,L,L,L
75,GIB,Gibraltar,UM,UM,UM,UM,UM,UM,UM,..,...,H,..,..,..,..,H,H,H,H,H
94,IMN,Isle of Man,H,H,H,UM,UM,UM,UM,UM,...,H,H,H,H,H,H,H,H,H,H
105,XKX,Kosovo,..,..,..,..,..,..,..,..,...,LM,LM,LM,LM,LM,LM,LM,LM,UM,UM
137,NRU,Nauru,..,..,..,..,..,..,..,..,...,..,..,..,..,..,H,UM,UM,UM,H
160,ROU,Romania,UM,UM,UM,LM,LM,LM,LM,LM,...,UM,UM,UM,UM,UM,UM,UM,UM,UM,H
190,TWN,"Taiwan, China",H,H,H,H,H,H,H,H,...,H,H,H,H,H,H,H,H,H,H
194,TLS,Timor-Leste,..,..,..,..,..,..,..,..,...,LM,LM,LM,LM,LM,LM,LM,LM,LM,LM


In [14]:
replace_country_codes = {'AND':'ADO', 
                         'COD':'ZAR', 
                         'IMN':'IMY',
                         'XKX':'KSV', 
                         'ROU':'ROM', 
                         'TLS':'TMP', 
                         'PSE':'WBG', 
                         'VGB': np.nan, 
                         'GIB': np.nan, 
                         'NRU': np.nan, 
                         'TWN': np.nan}

income_groups.country_code.replace(replace_country_codes, inplace=True)
income_groups.dropna(subset=['country_code'], inplace=True)
print(income_groups.shape)
income_groups.head()

(214, 35)


,country_code,country_name,1987,1988,1989,1990,1991,1992,1993,1994,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,AFG,Afghanistan,L,L,L,L,L,L,L,L,...,L,L,L,L,L,L,L,L,L,L
1,ALB,Albania,..,..,..,LM,LM,LM,L,L,...,UM,LM,UM,UM,UM,UM,UM,UM,UM,UM
2,DZA,Algeria,UM,UM,LM,LM,LM,LM,LM,LM,...,UM,UM,UM,UM,UM,UM,UM,UM,UM,LM
3,ASM,American Samoa,H,H,H,UM,UM,UM,UM,UM,...,UM,UM,UM,UM,UM,UM,UM,UM,UM,UM
4,ADO,Andorra,..,..,..,H,H,H,H,H,...,H,H,H,H,H,H,H,H,H,H


In [15]:
country_incomes = []
for row in income_groups.itertuples():
    country_code = row[1]
    name = row[2]
    year = 1987
    for i in range(33):
        country_incomes.append([country_code, name, year+i, row[3+i]])

income_group_country = pd.DataFrame(country_incomes, columns=['country_code', 'country_name', 'year', 'income_group'])
income_group_country.head()

,country_code,country_name,year,income_group
0,AFG,Afghanistan,1987,L
1,AFG,Afghanistan,1988,L
2,AFG,Afghanistan,1989,L
3,AFG,Afghanistan,1990,L
4,AFG,Afghanistan,1991,L


In [16]:
income_group_country.income_group.unique()

array(['L', '..', 'LM', 'UM', 'H', 'LM*'], dtype=object)

In [17]:
income_group_country.income_group.replace('LM*', 'LM', inplace=True)
income_group_country.income_group.unique()

array(['L', '..', 'LM', 'UM', 'H'], dtype=object)

In [18]:
## These are leading up to regions/countries being officially recognized as regions/countries. 
# Example: Armenia  was founded in 1991. The last value of '..' is in 1990. 
income_group_country[income_group_country.income_group == '..']

,country_code,country_name,year,income_group
33,ALB,Albania,1987,..
34,ALB,Albania,1988,..
35,ALB,Albania,1989,..
132,ADO,Andorra,1987,..
133,ADO,Andorra,1988,..
...,...,...,...,...
6932,WBG,West Bank and Gaza,1989,..
6933,WBG,West Bank and Gaza,1990,..
6934,WBG,West Bank and Gaza,1991,..
6935,WBG,West Bank and Gaza,1992,..


In [19]:
income_group_country.income_group.replace('..', np.nan, inplace=True)
print(income_group_country.isna().sum())

income_group_country.dropna(inplace=True)
income_group_country.reset_index(drop=True, inplace=True)
print(income_group_country.shape)

## matching up country names between in the income group table and country names that are in the indicator table
country_region_names = {i[2]:i[1] for i in countries_regions.itertuples()}

def fix_country_names(row):
    return country_region_names[row]

income_group_country.country_name = income_group_country.country_code.map(fix_country_names)
income_group_country.head()

country_code      0
country_name      0
year              0
income_group    392
dtype: int64
(6670, 4)


,country_code,country_name,year,income_group
0,AFG,Afghanistan,1987,L
1,AFG,Afghanistan,1988,L
2,AFG,Afghanistan,1989,L
3,AFG,Afghanistan,1990,L
4,AFG,Afghanistan,1991,L


In [20]:
indicator_types = indicators[['IndicatorName', 'IndicatorCode']].drop_duplicates().reset_index(drop=True)

indicator_first_occur = indicators.groupby('IndicatorCode').Year.min().reset_index()
indicator_first_occur.columns = ['IndicatorCode', 'FirstOccurance']

indicator_year_landmarks = indicator_first_occur.merge(
    indicators.groupby('IndicatorCode').Year.max().reset_index(), on='IndicatorCode'
)
indicator_year_landmarks.rename(columns={'Year':'LastOccurance'}, inplace=True)

indicator_types = indicator_types.merge(indicator_year_landmarks, on='IndicatorCode')
indicator_types.head()

,IndicatorName,IndicatorCode,FirstOccurance,LastOccurance
0,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1960,2014
1,Age dependency ratio (% of working-age populat...,SP.POP.DPND,1960,2014
2,"Age dependency ratio, old (% of working-age po...",SP.POP.DPND.OL,1960,2014
3,"Age dependency ratio, young (% of working-age ...",SP.POP.DPND.YG,1960,2014
4,Arms exports (SIPRI trend indicator values),MS.MIL.XPRT.KD,1960,2014


In [21]:
indicator_types['unit_value'] = indicator_types.IndicatorName.map(lambda x: re.findall('\(.+\)$', x))

indicator_types.unit_value = indicator_types.unit_value.map(
    lambda x: re.sub('[\(\)]', '', x[0]) if len(x) > 0 else np.nan
)

# some rows have the (%) sign in the middle of the text and the values that they are reporting are percents
percents = indicator_types[(indicator_types.unit_value.isna()) & 
                (indicator_types.IndicatorName.str.contains('%'))].IndicatorName.tolist()

indicator_types.iloc[269, -1] = 'tractors per 100 sq. km of arable land'

indicator_types.unit_value = indicator_types.apply(
    lambda x: '%' if x['IndicatorName'] in percents else x['unit_value'], 
    axis=1
)

indicator_types.unit_value = indicator_types.unit_value.replace(np.nan, 'Total/Count')

indicator_types.head()

,IndicatorName,IndicatorCode,FirstOccurance,LastOccurance,unit_value
0,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1960,2014,"births per 1,000 women ages 15-19"
1,Age dependency ratio (% of working-age populat...,SP.POP.DPND,1960,2014,% of working-age population
2,"Age dependency ratio, old (% of working-age po...",SP.POP.DPND.OL,1960,2014,% of working-age population
3,"Age dependency ratio, young (% of working-age ...",SP.POP.DPND.YG,1960,2014,% of working-age population
4,Arms exports (SIPRI trend indicator values),MS.MIL.XPRT.KD,1960,2014,SIPRI trend indicator values


In [22]:
indicator_types[indicator_types.unit_value.isna()]

,IndicatorName,IndicatorCode,FirstOccurance,LastOccurance,unit_value


In [29]:
# # # creating indicators table and inserting rows into table 
# create_indicator_table = """
# CREATE TABLE indicators (id SERIAL, 
#                          indicator_name VARCHAR, 
#                          indicator_code VARCHAR, 
#                          first_appearance INT NOT NULL, 
#                          last_appearance INT NOT NULL, 
#                          indicator_unit VARCHAR,
#                          PRIMARY KEY (indicator_code))
# """

# cur.execute(create_indicator_table)
# conn.commit()

# for row in indicator_types.itertuples():
#     cur.execute(f"""
#     INSERT INTO indicators (indicator_name, indicator_code, first_appearance, last_appearance, indicator_unit)
#     VALUES ('{row[1].replace("'", "")}', '{row[2]}', {row[3]}, {row[4]}, '{row[5]}')""")
    
# conn.commit()

# # creating country/region table and inserting rows into table
# create_countries_table = """
# CREATE TABLE countries_regions (id SERIAL, 
#                                 country_name TEXT NOT NULL, 
#                                 country_code TEXT NOT NULL,
#                                 PRIMARY KEY (country_code))
# """
# cur.execute(create_countries_table)
# conn.commit()

# for row in countries_regions.itertuples():
#     cur.execute(f"""
#     INSERT INTO countries_regions (country_name, country_code)
#     VALUES ('{row[1].replace("'", "")}', '{row[2]}')
#     """)
# conn.commit()

# # creating income group table and inserting rows into table b
# create_income_group_table = """
# CREATE TABLE income_groups (id SERIAL, 
#                             country_name TEXT NOT NULL, 
#                             country_code TEXT NOT NULL, 
#                             year INT NOT NULL, 
#                             income_group TEXT NOT NULL,
#                             PRIMARY KEY (id),
#                             FOREIGN KEY (country_code)
#                                 REFERENCES countries_regions(country_code))
# """

# cur.execute(create_income_group_table)
# conn.commit()

# for row in income_group_country.itertuples():
#     cur.execute(f"""
#     INSERT INTO income_groups (country_name, country_code, year, income_group)
#     VALUES ('{row[2].replace("'", "")}', '{row[1]}', {row[3]}, '{row[4]}')
#     """)
    
# create_overall_table = """
# CREATE TABLE indicator_by_country_year (id SERIAL,
#                                         country_code TEXT NOT NULL,
#                                         country_name TEXT NOT NULL,
#                                         indicator_name VARCHAR NOT NULL, 
#                                         indicator_code VARCHAR NOT NULL, 
#                                         year INT NOT NULL, 
#                                         value FLOAT NOT NULL,
#                                         PRIMARY KEY (id),
#                                         FOREIGN KEY (country_code)
#                                             REFERENCES countries_regions(country_code),
#                                         FOREIGN KEY (indicator_code)
#                                             REFERENCES indicators(indicator_code)
#                                             ON DELETE CASCADE
#                                             )
# """
# cur.execute(create_overall_table)
# conn.commit()

for row in tqdm(indicators.itertuples()):
    cur.execute(f"""
    INSERT INTO indicator_by_country_year (country_code, country_name, indicator_name, indicator_code, year, value)
    VALUES ('{row[2].replace("'", "")}', '{row[1].replace("'", "")}', '{row[3].replace("'", "")}', '{row[4]}', {row[5]}, {row[6]})
    """)
conn.commit()

5656458it [10:59, 8573.71it/s]
